## Домашнее задание к лекции «Случайные события. Случайные величины»

Смоделировать игру против лотерейного автомата типа “777”. Игрок платит 1 руб., после чего выпадает случайное целое число, равномерно распределенное от 0 до 999. При некоторых значениях числа игрок получает выигрыш (см. справа)

Выгодна ли игра игроку?

Сколько в среднем приобретает
или теряет игрок за одну игру?

777: 200 руб.  
999: 100 руб.  
555: 50 руб.  
333: 15 руб.  
111: 10 руб.  
\*77: 5 руб.  
\*\*7: 3 руб.  
\*00: 2 руб.  
\*\*0: 1 руб.

– любая цифра  

Дополнительное задание повышенной сложности. Теоретически рассчитать средний выигрыш (проигрыш) и сравнить с результатами моделирования

In [48]:
import re
import numpy as np

In [60]:
def get_winning(number):
    
    number = str(number)
    
    # если выпало не 3-значное число, приведем его к виду строки из 3 цифр, добавив необходимое кол-во 0 в начале
    if len(number) == 1:
        number = '00' + number
    elif len(number) == 2:
        number = '0' + number
        
    # сформулируем правила выигрыша
    # правила выигрыша меньших сумм должны учитывать правила выигрыша больших сумм, не пересекаться с ними
    # (например, 777 должно попадать под правило выигрыша 200руб., но не 5 руб.)
    
    rules_dict = {
        '777': 200,
        '999': 100,
        '555': 50,
        '333': 15,
        '111': 10,
        r'[^7]77': 5,
        r'\d[^7]7': 3, 
        r'\d00': 2, 
        r'\d[^0]0': 1
    }

    result = 0
    for rule in rules_dict:
        if re.match(rule, number):
            result = rules_dict[rule]
            
#     print(number, result)
    return result
        

In [64]:
# смоделируем игру
trials = 1000000
winnings= 0
losings = 0

for i in range(trials):
    number = np.random.randint(1, 1000)
    losings += 1
    winnings += get_winning(number)
    
print(f'в среднем выиграл {round(winnings/trials, 3)} руб., в среднем проиграл {losings/trials}')
print(f'в среднем приобрел за игру {round((winnings - losings)/trials, 3)}')
    

в среднем выиграл 0.801 руб., в среднем проиграл 1.0
в среднем приобрел за игру -0.199


Средний проигрыш при данных условиях всегда равен 1 руб. (стоимость 1й попытки). Средний выигрыш колеблется около значения 0,8, но ни разу не превосходил 1. Т.е. получаем, что по итогам моделирования средний выигрыш меньше, чем средний проигрыш. Потеря от 1й игры состовляет примерно 0,2 руб. Следовательно **игра не выгодна для игрока**. В выигрыше казино, как и всегда

### Попробуем рассчитать средний выигрыш теоретически

Очевидно, что вероятность выиграть суммы 200, 100, 50, 15 и 10 руб. составляют 1/1000 каждая. Т.к. мы ожидаем встретить одно конкретное число из тысячи при равномерном распределении

Комбинации **\*77** соответствует 10 чисел из тысячи. Но здесь мы должны выкинуть из рассмотрения 777, т.к. оно соответствует другому правилу выигрыша. Следовательно, вероятность выиграть 5 руб. составляет 9/1000

Комбинации **\*\*7** соответствует 100 чисел из тысячи. Но здесь мы должны выкинуть из рассмотрения 777 и 9 чисел из предыдущего пункта. Значит, имеем шанс 90/1000 выиграть 3 руб.
 
Комбинации **\*00** соответствует 10 чисел из тысячи. Это правило с предыдущими не пересекается, значит, имеем вероятность 10/1000 выиграть 2 руб.
 
Наконец, комбинации **\*\*0** соответствует 100 чисел из тысячи. Здесьмы должны выкинуть из рассмотрения 10 чисел, соответствующих предыдущему правилу. Значит вероятность выиграть 1 руб. составляет 90/1000
 
 Посчитаем средний ожидаемый выигрыш с учетом вероятностей, приведенных выше:

In [65]:
winning = 200*(1/1000) + 100*(1/1000) + 50*(1/1000) + 15*(1/1000) + 10*(1/1000) + 5*(9/1000) + 3*(90/1000) + 2*(10/1000) + (90/1000)
winning

0.8

Получили теоретический средний выигрыш от одного хода = 0,8 руб. Это меньше стоимости 1й попытки, соответственно, это подтверждает идею о том, что игра не выгодна для игрока, и что вероятнее он будет терять деньги, а не заработает. Так же полученный результат хорошо соотносится с результатом моделирования. Почти совпадает с ним при большом количестве попыток.  